In [1]:
%load_ext autoreload
# %reload_ext autoreload

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from pycaret.regression import *
from pathlib import Path

In [3]:
# from pycaret.utils import version
# version()

In [4]:
%autoreload
import sys
sys.path.append('../')
from pipeline import Pipin
pipin = Pipin()

Current working directory: /Users/matsalexander/Desktop/Forest Gump/mats/pycret


In [5]:
df1_0 = pipin.get_data("A")
df2_0 = pipin.get_data("B")
df3_0 = pipin.get_data("C")

In [6]:
# can you move the pv_measurment to the end of the df?
df1 = df1_0.copy()
df2 = df2_0.copy()
df3 = df3_0.copy()
categorical_features = pipin.get_categorical_features(df1)
ignore_features = pipin.get_irrelevant_features()


In [7]:
# add weights
# if is 
df1['weight'] = df1['is_estimated:idx'] * 2 + 1
df2['weight'] = df2['is_estimated:idx'] * 2 + 1
df3['weight'] = df3['is_estimated:idx'] * 2 + 1

In [8]:
# ignore_features.append('weight')
# s = setup(df_transformed, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False)
s1 = setup(df1, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, train_size=0.9, fold_shuffle=True)
s2 = setup(df2, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, train_size=0.9, fold_shuffle=True)
s3 = setup(df3, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, train_size=0.9, fold_shuffle=True)
# s = setup(df_transformed, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, imputation_type="iterative", feature_selection="classic", feature_selection_estimator="et", remove_multicollinearity=True, remove_outliers = True, low_variance_threshold = 0.1,)
# s = setup(df_transformed, target="pv_measurement", imputation_type="iterative", feature_selection="classic", feature_selection_estimator="et", remove_multicollinearity=True, remove_outliers = True, low_variance_threshold = 0.1,  normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False,)

,Description,Value
0,Session id,1945
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(21282, 50)"
4,Transformed data shape,"(21282, 54)"
5,Transformed train set shape,"(19153, 54)"
6,Transformed test set shape,"(2129, 54)"
7,Ignore features,3
8,Ordinal features,3
9,Numeric features,41


,Description,Value
0,Session id,8065
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(14757, 50)"
4,Transformed data shape,"(14757, 53)"
5,Transformed train set shape,"(13281, 53)"
6,Transformed test set shape,"(1476, 53)"
7,Ignore features,3
8,Ordinal features,3
9,Numeric features,41


,Description,Value
0,Session id,5604
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(11993, 50)"
4,Transformed data shape,"(11993, 53)"
5,Transformed train set shape,"(10793, 53)"
6,Transformed test set shape,"(1200, 53)"
7,Ignore features,3
8,Ordinal features,3
9,Numeric features,41


In [9]:
# # now pass sample weights to lr and fit again
# import numpy as np
# # weights = np.arange(s1.get_config('X_train').shape[0])
# et_fit = s1.create_model('lr')


In [11]:
# weights = list(s1.train_transformed["weight"])
# print(len(weights))
# et_fit.fit(X = s1.train_transformed, y = s1.test_transformed, sample_weight = weights)
# # weights

In [12]:
preprocessed_data = s1.dataset_transformed
preprocessed_data
print(preprocessed_data.shape)
print (preprocessed_data.columns)
preprocessed_data.head(20)
# preprocessed_data.describe()
# categorical features are normalized. That is not good.

(21282, 54)
Index(['absolute_humidity_2mgm3', 'air_density_2mkgm3', 'ceiling_height_aglm',
       'clear_sky_energy_1hJ', 'clear_sky_radW', 'cloud_base_aglm',
       'dew_or_rimeidx_0.0', 'dew_or_rimeidx_1.0', 'dew_or_rimeidx_-1.0',
       'dew_point_2mK', 'diffuse_radW', 'diffuse_rad_1hJ', 'direct_radW',
       'direct_rad_1hJ', 'effective_cloud_coverp', 'elevationm',
       'fresh_snow_12hcm', 'fresh_snow_1hcm', 'fresh_snow_24hcm',
       'fresh_snow_3hcm', 'fresh_snow_6hcm', 'is_dayidx', 'is_in_shadowidx',
       'msl_pressurehPa', 'precip_5minmm', 'precip_type_5minidx_0.0',
       'precip_type_5minidx_1.0', 'precip_type_5minidx_3.0',
       'precip_type_5minidx_2.0', 'precip_type_5minidx_5.0',
       'pressure_100mhPa', 'pressure_50mhPa', 'prob_rimep', 'rain_waterkgm2',
       'relative_humidity_1000hPap', 'sfc_pressurehPa', 'snow_densitykgm3',
       'snow_depthcm', 'snow_driftidx_0.0', 'snow_melt_10minmm',
       'snow_waterkgm2', 'sun_azimuthd', 'sun_elevationd',
       'super_c

,absolute_humidity_2mgm3,air_density_2mkgm3,ceiling_height_aglm,clear_sky_energy_1hJ,clear_sky_radW,cloud_base_aglm,dew_or_rimeidx_0.0,dew_or_rimeidx_1.0,dew_or_rimeidx_-1.0,dew_point_2mK,...,t_1000hPaK,total_cloud_coverp,visibilitym,wind_speed_10mms,wind_speed_u_10mms,wind_speed_v_10mms,wind_speed_w_1000hPams,is_estimatedidx,weight,pv_measurement
0,7.9,1.258,9051.200195,0.000000e+00,0.000000,4265.399902,1.0,0.0,0.0,280.500000,...,285.000000,100.000000,40184.699219,1.6,-1.3,0.8,-0.0,0.0,1.0,0.000000
1,13.5,1.208,649.500000,1.427213e+05,14.500000,649.500000,1.0,0.0,0.0,289.100006,...,290.899994,20.900000,42677.500000,1.4,1.3,0.4,-0.0,0.0,1.0,14.960000
2,10.7,1.215,483.700012,1.095320e+06,369.500000,113.699997,1.0,0.0,0.0,285.299988,...,285.700012,100.000000,919.700012,5.1,5.0,-0.7,-0.0,0.0,1.0,299.859985
3,8.1,1.255,491.299988,1.185852e+05,14.300000,347.899994,1.0,0.0,0.0,280.899994,...,281.799988,64.099998,31236.099609,5.4,5.3,0.5,-0.0,1.0,3.0,1.100000
4,8.5,1.246,966.200012,1.636200e+03,5.800000,966.200012,1.0,0.0,0.0,281.600006,...,282.399994,58.900002,28881.300781,1.3,1.1,0.7,-0.0,0.0,1.0,69.959999
5,9.3,1.206,2605.199951,2.371514e+06,618.299988,2746.100098,1.0,0.0,0.0,283.399994,...,287.700012,100.000000,24405.300781,0.7,0.0,0.7,-0.0,0.0,1.0,1034.880005
6,4.3,1.266,4384.600098,2.500561e+05,121.800003,320.500000,1.0,0.0,0.0,271.700012,...,274.100006,98.300003,29519.699219,3.1,2.3,-2.1,0.0,0.0,1.0,633.159973
7,8.8,1.219,1027.199951,2.468712e+05,116.800003,1027.199951,1.0,0.0,0.0,282.200012,...,284.299988,100.000000,4917.299805,2.9,2.3,1.7,-0.0,0.0,1.0,161.479996
8,6.4,1.258,401.600006,4.765920e+04,0.000000,378.799988,1.0,0.0,0.0,277.200012,...,278.799988,91.000000,35970.500000,1.4,-0.0,1.4,-0.0,0.0,1.0,0.000000
9,7.3,1.254,1194.099976,1.633588e+06,444.799988,1152.000000,1.0,0.0,0.0,279.399994,...,282.000000,100.000000,13216.700195,0.7,0.0,-0.7,-0.0,0.0,1.0,983.400024


In [13]:
# top3 = s1.compare_models(sort='MAE', n_select=3, )
gmb1 = s1.create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,305.1292,275372.7867,524.7597,0.8431,1.1903,2.1196
1,305.0861,288175.7687,536.8201,0.8374,1.2261,2.9749
2,290.9406,268006.6599,517.6936,0.8511,1.2836,2.7529
3,295.3855,265801.2723,515.5592,0.8495,1.1858,3.5213
4,313.5913,304712.5528,552.0077,0.8334,1.1920,2.9740
5,308.4445,279484.7401,528.6632,0.8485,1.2347,3.0882
6,296.3349,266597.6743,516.3310,0.8492,1.2249,2.9320
7,295.1606,257599.2182,507.5423,0.8521,1.2217,2.5293
8,293.6579,267862.5555,517.5544,0.8483,1.2153,2.3855


In [16]:
gbm1 = s1.tune_model(gmb1, optimize='MAE', n_iter=50, choose_better=True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,292.9453,244141.8093,494.1071,0.8609,1.4018,2.8983
1,309.8007,287605.8852,536.2890,0.8378,1.4614,3.4229
2,296.3348,270215.7602,519.8228,0.8499,1.4976,2.9176
3,297.5619,270903.8744,520.4843,0.8466,1.4001,4.7514
4,307.6381,284516.3760,533.4008,0.8444,1.3487,3.6733
5,297.7842,266554.4107,516.2891,0.8555,1.3566,3.2252
6,293.9025,262027.3141,511.8860,0.8518,1.3821,2.6243
7,295.3019,257159.4354,507.1089,0.8524,1.3941,2.6668
8,302.1023,279439.7672,528.6206,0.8417,1.4615,3.0993


Fitting 10 folds for each of 50 candidates, totalling 500 fits
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightG

In [17]:
# et1 = s1.create_model('et')

In [18]:
# top3 = s2.compare_models(sort='MAE', n_select=3)
et2 = s2.create_model('et')
# lightgbm = s.create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,49.0080,7833.3187,88.5060,0.8735,0.7907,1.8360
1,44.8199,6916.4947,83.1655,0.8940,0.7781,2.0113
2,49.2505,7666.5112,87.5586,0.8798,0.7862,2.2593
3,50.8881,8185.6209,90.4744,0.8699,0.7848,1.4639
4,47.2524,7654.3802,87.4893,0.8727,0.7979,1.9122
5,46.7396,7633.0949,87.3676,0.8767,0.7335,7.2226
6,47.3874,7330.4332,85.6179,0.8892,0.7362,1.6996
7,44.4481,6172.4713,78.5651,0.9133,0.7654,1.2737
8,48.2739,7430.6493,86.2012,0.8855,0.8086,1.8812


In [19]:
et2 = s2.tune_model(et2, optimize='MAE', n_iter=50, choose_better=True)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:29:18
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 50 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
# top3 = s3.compare_models(sort='MAE', n_select=3)
et3 = s3.create_model('et')

# lightgbm = s.create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,41.8051,5082.9030,71.2945,0.8851,0.6511,0.6772
1,43.7563,5791.6608,76.1030,0.8769,0.6400,0.8570
2,39.9690,5326.4220,72.9823,0.8721,0.6197,0.6143
3,41.2518,4784.6401,69.1711,0.8844,0.6387,0.6766
4,41.5609,5372.5217,73.2975,0.8842,0.6697,0.8010
5,41.9760,5705.1374,75.5324,0.8806,0.6129,0.6255
6,39.5368,4937.3311,70.2661,0.8899,0.6551,0.7100
7,43.3221,5432.2582,73.7039,0.8779,0.6673,0.6317
8,42.7923,5164.6993,71.8658,0.8825,0.7204,0.7112


In [ ]:
et3 = s3.tune_model(et3, optimize='MAE', n_iter=50, choose_better=True)

In [ ]:

# et3 = s3.tune_model(et3, optimize="MAE")

In [ ]:
et1 = s1.finalize_model(gmb1)

In [ ]:
et2 = s2.finalize_model(et2)


In [ ]:
et3 = s3.finalize_model(et3)

In [ ]:
# plot_model(lightgbm, plot="feature_all")
# plot_model(gump, plot="feature_all")

In [ ]:
test1 = pipin.get_test_data("A")
test2 = pipin.get_test_data("B")
test3 = pipin.get_test_data("C")

In [ ]:
from utils import get_unique_filename
pred1 = predict_model(et1, data=test1)
pred2 = predict_model(et2, data=test2)
pred3 = predict_model(et3, data=test3)
pred = pd.concat([pred1, pred2, pred3])
final_prediction = pipin.post_processing(pred)
# final_prediction.to_csv(get_unique_filename('et3x'))
final_prediction


,prediction
id,
0,0.015400
1,0.180400
2,5.253600
3,177.806200
4,528.867896
...,...
2155,74.568200
2156,24.754800
2157,10.172400


In [ ]:
# print("done", res_stack.shape)
# pipin.compare_mae(res_stack)
